In [1]:
import pandas as pd
import numpy as np
from cv_py import CrossValidation
from dataloading import DataLoader
from eda_py import EDA
from visualize_py import Visualize
import seaborn as sns
import matplotlib.pyplot as plt
from implicit.evaluation import train_test_split
from eals import ElementwiseAlternatingLeastSquares, load_model

In [2]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [3]:
dl = DataLoader()
user_item_co = dl.import_data('TEREX', 'CO', 'df', clip=99.999)
user_item_po = dl.import_data('TEREX', 'PO', 'df', clip=99.999)

In [4]:
user_item_csr = dl.to_csr(user_item_co)

In [5]:
train, test = train_test_split(user_item_csr, train_percentage=0.9)

In [6]:
cv = CrossValidation(train, 5)

In [7]:
test_k, train_k = cv.split_k_fold()

In [9]:
alpha = (user_item_csr.shape[0] * user_item_csr.shape[1] - user_item_csr.nnz) / sum(user_item_csr.data)

In [10]:
alpha

1.0920634173446777

In [8]:
space_iALS = {'factors' : [64], 'regularization' : [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5], 'alpha' : [0.5, 1, 2, 5, 10, 15, 20, 30], 'iterations' : [15]}
hyper_test_als = cv.hyperp_tuning(test_k, train_k, space_iALS, 'iALS')


c:\Users\781110104\Anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


In [13]:
hyper_test_als

,factors,regularization,alpha,iterations,precision,map,ndcg,auc,mpr
0,64,0.0,0.5,15,0.6490,0.599128,0.696918,0.522382,0.366829
1,64,0.0,1.0,15,0.7130,0.663221,0.750727,0.524281,0.324983
2,64,0.0,2.0,15,0.7450,0.700125,0.776983,0.525038,0.293128
3,64,0.0,5.0,15,0.7570,0.708347,0.783899,0.524884,0.259729
4,64,0.0,10.0,15,0.7695,0.718493,0.794917,0.525028,0.244586
...,...,...,...,...,...,...,...,...,...
75,64,5.0,5.0,15,0.7505,0.688509,0.776494,0.525422,0.195689
76,64,5.0,10.0,15,0.7430,0.661702,0.748823,0.525902,0.208588
77,64,5.0,15.0,15,0.7415,0.660079,0.746799,0.525885,0.206358
78,64,5.0,20.0,15,0.7315,0.639483,0.730781,0.525728,0.212681


In [13]:
space_LMF = {'factors' : [64], 'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 2, 5], 'regularization' : [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5], 'iterations' : [35], 'neg_prop' : [30]}
hyper_test_lmf = cv.hyperp_tuning(test_k, train_k, space_LMF, 'LMF')

{'factors': 64, 'learning_rate': 0.001, 'regularization': 0, 'iterations': 35, 'neg_prop': 30}


In [14]:
space_LMF = {'factors' : [64], 'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 2, 5], 'regularization' : [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5], 'iterations' : [35], 'neg_prop' : [5, 10, 20]}
hyper_test_lmf_add = cv.hyperp_tuning(test_k, train_k, space_LMF, 'LMF')

In [45]:
hyper_test_lmf

,factors,learning_rate,regularization,iterations,neg_prop,precision,map,ndcg,auc,mpr
0,64,1.5,0.1,35,30,0.2805,0.219318,0.312168,0.506108,0.397814


In [9]:
space_BPR = {'factors' : [64], 'learning_rate': [0.001, 0.005, 0.01, 0.03, 0.07, 0.1, 0.2, 0.3], 'regularization' : [0.001, 0.005, 0.01, 0.03, 0.07, 0.1, 0.12], 'iterations' : [100]}
hyper_test_bpr = cv.hyperp_tuning(test_k, train_k, space_BPR, 'BPR')

In [12]:
space_eALS = {'factors' : [64], 'alpha': [0.25], 'regularization':[0.001], 'w0':[5]}
test_cv_eals = cv.hyperp_tuning(test_k, train_k, space_eALS, 'eALS')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>


In [15]:
space_eALS = {'factors' : [64], 'alpha': [0.25, 0.5, 0.75], 'regularization':[0.001, 0.005, 0.01, 0.05, 0.1, 0.5], 'w0':[5, 10, 20, 30]}
test_cv_eals = cv.hyperp_tuning(test_k, train_k, space_eALS, 'eALS')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [16]:
hyper_test_als.to_csv('als_tuning.csv')
hyper_test_bpr.to_csv('bpr_tuning.csv')
pd.concat((hyper_test_lmf, hyper_test_lmf_add), axis=0).to_csv('lmf_tuning.csv')
test_cv_eals.to_csv('eals_tuning.csv')